In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report
import re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

In [4]:
# Create a sample support ticket dataset
ticket_data = {
    'text': [
        "My login password isn't working, I need to reset it",
        "The application is running very slow today",
        "I can't access my files, getting permission denied error",
        "How do I update my billing information?",
        "The website is down and showing error 504",
        "I need help installing the software on my new computer",
        "My invoice amount seems incorrect, can you check?",
        "How do I export my data to CSV format?",
        "I'm getting a database connection error when trying to save",
        "Can you help me understand the pricing plans?",
        "The upload feature is not working, files fail to upload",
        "I want to cancel my subscription and get a refund",
        "How do I integrate with the API?",
        "The dashboard is not loading properly in my browser",
        "I need to change my account email address",
        "The mobile app crashes when I try to open it",
        "Where can I find the user manual?",
        "I'm getting a 'payment failed' notification but my card is valid",
        "How do I share my project with team members?",
        "The report generation is taking too long to complete"
    ],
    'label': [
        'login', 'performance', 'access', 'billing', 'server',
        'installation', 'billing', 'how-to', 'technical', 'sales',
        'technical', 'billing', 'how-to', 'technical', 'account',
        'technical', 'documentation', 'billing', 'how-to', 'performance'
    ]
}

# Convert to DataFrame
df = pd.DataFrame(ticket_data)

# Define our categories
categories = ['login', 'performance', 'access', 'billing', 'technical', 'how-to', 'account', 'sales', 'documentation', 'server']

print("Dataset shape:", df.shape)
print("\nCategory distribution:")
print(df['label'].value_counts())
print("\nSample tickets:")
df.head(10)

Dataset shape: (20, 2)

Category distribution:
label
billing          4
technical        4
how-to           3
performance      2
login            1
access           1
server           1
installation     1
sales            1
account          1
documentation    1
Name: count, dtype: int64

Sample tickets:


,text,label
0,"My login password isn't working, I need to res...",login
1,The application is running very slow today,performance
2,"I can't access my files, getting permission de...",access
3,How do I update my billing information?,billing
4,The website is down and showing error 504,server
5,I need help installing the software on my new ...,installation
6,"My invoice amount seems incorrect, can you check?",billing
7,How do I export my data to CSV format?,how-to
8,I'm getting a database connection error when t...,technical
9,Can you help me understand the pricing plans?,sales


In [5]:
"""## Step 2: Set Up the LLM

We'll use a pre-trained model from Hugging Face for zero-shot classification.
"""

# Load a zero-shot classification pipeline
classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

# Test the classifier with a sample
sample_text = "I can't log in to my account"
result = classifier(sample_text, categories, multi_label=False)
print("Sample classification result:")
print(f"Text: {sample_text}")
print(f"Predicted: {result['labels'][0]} (confidence: {result['scores'][0]:.4f})")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Sample classification result:
Text: I can't log in to my account
Predicted: account (confidence: 0.3666)


In [6]:
"""## Step 3: Implement Zero-Shot Classification"""

def zero_shot_classify(text, candidate_labels):
    """
    Classify text using zero-shot learning
    """
    result = classifier(text, candidate_labels, multi_label=False)
    return result['labels'], result['scores']

# Test on our dataset
test_text = df['text'].iloc[0]
labels, scores = zero_shot_classify(test_text, categories)

print(f"Text: {test_text}")
print("Top 3 predictions:")
for i in range(3):
    print(f"{i+1}. {labels[i]} (confidence: {scores[i]:.4f})")


Text: My login password isn't working, I need to reset it
Top 3 predictions:
1. login (confidence: 0.4281)
2. access (confidence: 0.2517)
3. technical (confidence: 0.1243)


In [7]:
"""## Step 4: Implement Few-Shot Classification

For few-shot learning, we'll provide examples to help the model understand the categories better.
"""

# Create few-shot examples for each category
few_shot_examples = {
    'login': [
        "I forgot my password and need to reset it",
        "My login credentials are not working",
        "I'm locked out of my account"
    ],
    'performance': [
        "The application is running very slow",
        "Pages are taking too long to load",
        "Response time is very high today"
    ],
    'access': [
        "I can't access my files",
        "Permission denied when trying to open document",
        "Getting access error for shared folder"
    ],
    'billing': [
        "I need to update my payment method",
        "My invoice amount seems incorrect",
        "How do I change my billing address?"
    ],
    'technical': [
        "The application keeps crashing",
        "I'm getting error messages when saving",
        "The feature is not working as expected"
    ],
    'how-to': [
        "How do I export my data?",
        "Where can I find the settings menu?",
        "How to share projects with team members?"
    ],
    'account': [
        "I need to change my email address",
        "How do I delete my account?",
        "Can you update my account information?"
    ],
    'sales': [
        "I have questions about pricing plans",
        "What features are included in the enterprise plan?",
        "Can I get a demo of the premium features?"
    ],
    'documentation': [
        "Where can I find the user manual?",
        "Is there a guide for API integration?",
        "I need documentation for advanced features"
    ],
    'server': [
        "The website is down",
        "Server is not responding",
        "Getting 500 error when accessing the service"
    ]
}

def create_few_shot_prompt(text, examples_dict, top_n=3):
    """
    Create a prompt with few-shot examples for classification
    """
    prompt = "Classify the following support ticket into one of these categories: "
    prompt += ", ".join(examples_dict.keys()) + ".\n\n"

    # Add examples
    for category, examples in examples_dict.items():
        for example in examples[:2]:  # Use 2 examples per category
            prompt += f"Example: {example}\nCategory: {category}\n\n"

    prompt += f"Now classify this ticket: {text}\nCategory:"
    return prompt

# Test the few-shot prompt
test_text = "I need to reset my password"
prompt = create_few_shot_prompt(test_text, few_shot_examples)
print("Few-shot prompt example:")
print(prompt)


Few-shot prompt example:
Classify the following support ticket into one of these categories: login, performance, access, billing, technical, how-to, account, sales, documentation, server.

Example: I forgot my password and need to reset it
Category: login

Example: My login credentials are not working
Category: login

Example: The application is running very slow
Category: performance

Example: Pages are taking too long to load
Category: performance

Example: I can't access my files
Category: access

Example: Permission denied when trying to open document
Category: access

Example: I need to update my payment method
Category: billing

Example: My invoice amount seems incorrect
Category: billing

Example: The application keeps crashing
Category: technical

Example: I'm getting error messages when saving
Category: technical

Example: How do I export my data?
Category: how-to

Example: Where can I find the settings menu?
Category: how-to

Example: I need to change my email address
Categor

In [8]:
"""## Step 5: Compare Zero-Shot vs Few-Shot Performance

We'll create a more advanced function that can use both approaches.
"""

def classify_ticket(text, candidate_labels, method='zero-shot', examples_dict=None):
    """
    Classify a support ticket using either zero-shot or few-shot approach
    Returns top 3 predicted labels with confidence scores
    """
    if method == 'zero-shot':
        # Use the zero-shot classifier
        result = classifier(text, candidate_labels, multi_label=False)
        top_labels = result['labels'][:3]
        top_scores = result['scores'][:3]
    else:
        # For few-shot, we'll use a simpler approach since we don't have a dedicated few-shot model
        # We'll use the zero-shot classifier but with a more specific hypothesis
        hypotheses = [f"This ticket is about {label}" for label in candidate_labels]
        result = classifier(text, candidate_labels, hypothesis_template="This ticket is about {}", multi_label=False)
        top_labels = result['labels'][:3]
        top_scores = result['scores'][:3]

    return top_labels, top_scores

# Test both approaches
test_text = "I can't access the server, it's showing error 503"

print(f"Text: {test_text}\n")

# Zero-shot
labels_zs, scores_zs = classify_ticket(test_text, categories, 'zero-shot')
print("Zero-shot results:")
for i, (label, score) in enumerate(zip(labels_zs, scores_zs)):
    print(f"{i+1}. {label} ({score:.4f})")

print()

# Few-shot
labels_fs, scores_fs = classify_ticket(test_text, categories, 'few-shot', few_shot_examples)
print("Few-shot results:")
for i, (label, score) in enumerate(zip(labels_fs, scores_fs)):
    print(f"{i+1}. {label} ({score:.4f})")


Text: I can't access the server, it's showing error 503

Zero-shot results:
1. server (0.4994)
2. access (0.2459)
3. technical (0.1196)

Few-shot results:
1. access (0.3326)
2. server (0.2846)
3. technical (0.1667)


In [10]:
"""## Step 6: Evaluate on the Entire Dataset"""

def evaluate_classification(df, candidate_labels, method='zero-shot'):
    """
    Evaluate the classification performance on the entire dataset
    """
    true_labels = []
    pred_labels = []
    all_top3 = []

    for i, row in df.iterrows():
        text = row['text']
        true_label = row['label']

        # Get prediction
        top_labels, top_scores = classify_ticket(text, candidate_labels, method)
        pred_label = top_labels[0]  # Top prediction

        true_labels.append(true_label)
        pred_labels.append(pred_label)
        all_top3.append(top_labels)

    # Calculate accuracy
    accuracy = accuracy_score(true_labels, pred_labels)

    # Check if true label is in top 3 predictions
    top3_accuracy = sum(1 for i, true_label in enumerate(true_labels)
                       if true_label in all_top3[i]) / len(true_labels)

    print(f"Method: {method}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Top-3 Accuracy: {top3_accuracy:.4f}")
    print("\nClassification Report:")

    # Get all unique labels from both true and predicted labels
    all_unique_labels = sorted(list(set(true_labels + pred_labels)))

    print(classification_report(true_labels, pred_labels, labels=all_unique_labels, target_names=all_unique_labels))

    return accuracy, top3_accuracy

# Evaluate zero-shot
print("Zero-shot evaluation:")
zs_accuracy, zs_top3 = evaluate_classification(df, categories, 'zero-shot')
print("\n" + "="*50 + "\n")

# Evaluate few-shot
print("Few-shot evaluation:")
fs_accuracy, fs_top3 = evaluate_classification(df, categories, 'few-shot')

Zero-shot evaluation:
Method: zero-shot
Accuracy: 0.5500
Top-3 Accuracy: 0.8500

Classification Report:
               precision    recall  f1-score   support

       access       1.00      1.00      1.00         1
      account       0.50      1.00      0.67         1
      billing       0.75      0.75      0.75         4
documentation       0.50      1.00      0.67         1
       how-to       1.00      0.33      0.50         3
 installation       0.00      0.00      0.00         1
        login       1.00      1.00      1.00         1
  performance       0.33      0.50      0.40         2
        sales       0.00      0.00      0.00         1
       server       0.00      0.00      0.00         1
    technical       0.33      0.50      0.40         4

     accuracy                           0.55        20
    macro avg       0.49      0.55      0.49        20
 weighted avg       0.55      0.55      0.51        20



Few-shot evaluation:
Method: few-shot
Accuracy: 0.6500
Top-3 Accur

In [11]:
"""## Step 7: Create a Function for Top 3 Tags"""

def get_top3_tags(text, candidate_labels, method='zero-shot'):
    """
    Get top 3 most probable tags for a support ticket
    """
    top_labels, top_scores = classify_ticket(text, candidate_labels, method)

    result = []
    for label, score in zip(top_labels, top_scores):
        result.append({
            'tag': label,
            'confidence': float(score),
            'percentage': f"{score*100:.1f}%"
        })

    return result

# Test the function
test_text = "The application is very slow and keeps freezing"
top3_tags = get_top3_tags(test_text, categories, 'zero-shot')

print(f"Text: {test_text}")
print("\nTop 3 predicted tags:")
for i, tag_info in enumerate(top3_tags):
    print(f"{i+1}. {tag_info['tag']} - {tag_info['percentage']} confidence")

Text: The application is very slow and keeps freezing

Top 3 predicted tags:
1. performance - 67.1% confidence
2. technical - 18.8% confidence
3. account - 4.7% confidence


In [17]:
"""## Step 8: Create a Streamlit App for Deployment"""

# Create a Streamlit app file
#%%writefile support_ticket_app.py

import streamlit as st
import pandas as pd
from transformers import pipeline
import numpy as np

# Set page config
st.set_page_config(
    page_title="Support Ticket Auto-Tagging",
    page_icon="🏷️",
    layout="wide"
)

# App title and description
st.title("🏷️ Support Ticket Auto-Tagging")
st.write("This app automatically tags support tickets into categories using a large language model.")

# Load the zero-shot classifier
@st.cache_resource
def load_classifier():
    return pipeline(
        "zero-shot-classification",
        model="facebook/bart-large-mnli"
    )

classifier = load_classifier()

# Define categories
categories = [
    'login', 'performance', 'access', 'billing', 'technical',
    'how-to', 'account', 'sales', 'documentation', 'server'
]

# Function to classify text
def classify_ticket(text, candidate_labels):
    result = classifier(text, candidate_labels, multi_label=False)
    top_labels = result['labels'][:3]
    top_scores = result['scores'][:3]

    result = []
    for label, score in zip(top_labels, top_scores):
        result.append({
            'tag': label,
            'confidence': float(score),
            'percentage': f"{score*100:.1f}%"
        })

    return result

# Input section
st.header("Tag a Support Ticket")
ticket_text = st.text_area(
    "Enter the support ticket text:",
    "I can't log in to my account and need to reset my password",
    height=100
)

# Classification method
method = st.radio(
    "Classification method:",
    ["Zero-shot", "Few-shot (simulated)"],
    help="Zero-shot uses no examples, Few-shot uses example tickets for each category"
)

# Classify button
if st.button("Tag Ticket"):
    if not ticket_text.strip():
        st.error("Please enter some text to classify.")
    else:
        with st.spinner("Analyzing ticket..."):
            # Get classification results
            top_tags = classify_ticket(ticket_text, categories)

            # Display results
            st.subheader("Predicted Tags")

            col1, col2, col3 = st.columns(3)

            with col1:
                tag = top_tags[0]
                st.metric("Primary Tag", tag['tag'])
                st.write(f"Confidence: {tag['percentage']}")
                st.progress(float(tag['confidence']))

            with col2:
                tag = top_tags[1]
                st.metric("Secondary Tag", tag['tag'])
                st.write(f"Confidence: {tag['percentage']}")
                st.progress(float(tag['confidence']))

            with col3:
                tag = top_tags[2]
                st.metric("Tertiary Tag", tag['tag'])
                st.write(f"Confidence: {tag['percentage']}")
                st.progress(float(tag['confidence']))

            # Show all tags in a table
            st.subheader("All Tag Probabilities")
            tags_df = pd.DataFrame(top_tags)
            st.dataframe(tags_df[['tag', 'percentage']])

# Add examples section
st.header("Example Tickets")
example_tickets = {
    "Login issue": "I forgot my password and can't access my account",
    "Performance problem": "The application is running very slow today",
    "Billing question": "How do I update my credit card information?",
    "Technical error": "I'm getting a 404 error when trying to save my work",
    "How-to question": "Where can I find the export functionality?"
}

example = st.selectbox("Choose an example ticket:", list(example_tickets.keys()))
if example:
    st.text_area("Example text:", example_tickets[example], height=60)

# Model info section
st.sidebar.header("About the Model")
st.sidebar.info(
    "This app uses a zero-shot classification model based on Facebook's BART-large-MNLI. "
    "It can classify text into predefined categories without any task-specific training."
)

# Category definitions
st.sidebar.header("Category Definitions")
category_descriptions = {
    'login': "Issues related to authentication, passwords, and account access",
    'performance': "Problems with speed, responsiveness, or resource usage",
    'access': "Permission and accessibility issues",
    'billing': "Questions about payments, invoices, and subscriptions",
    'technical': "Software bugs, errors, and technical problems",
    'how-to': "Questions about how to use features or functionality",
    'account': "Account management and profile changes",
    'sales': "Questions about products, pricing, and plans",
    'documentation': "Requests for guides, manuals, and documentation",
    'server': "Server availability and connectivity issues"
}

for category, description in category_descriptions.items():
    st.sidebar.write(f"**{category}**: {description}")

# Footer
st.markdown("---")
st.markdown("Built with 🤍 using Transformers and Streamlit")

2025-08-26 05:48:04.980 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 05:48:04.981 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 05:48:05.143 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-08-26 05:48:05.144 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 05:48:05.145 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 05:48:05.146 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 05:48:05.147 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

DeltaGenerator()

In [20]:
!streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.87.186.104:8501

  Stopping...
  Stopping...
